In [22]:
import org.apache.spark.sql._
import spark.implicits._

import org.apache.spark.sql._
import spark.implicits._


In [33]:
val col_names = List("age", "workclass", "fnlwgt", "education", "education-num","marital-status", "occupation", 
             "relationship", "race", "sex", "capital-gain", "capital-loss", "hours-per-week", 
             "native-country", "earnings")

col_names: List[String] = List(age, workclass, fnlwgt, education, education-num, marital-status, occupation, relationship, race, sex, capital-gain, capital-loss, hours-per-week, native-country, earnings)


In [49]:
val data = spark.read.options(
Map("header" -> "false", "inferSchema" -> "true", "ignoreLeadingWhiteSpace" -> "true")
).csv("adult.data")

data: org.apache.spark.sql.DataFrame = [_c0: int, _c1: string ... 13 more fields]


In [64]:
val column_replacing = data.columns.zip(col_names).map{case (old, nevv) => col(old).alias(nevv)}
var df = data.select(column_replacing: _*).drop(col("fnlwgt")).na.drop("any")

df.show(2,1000, true)

//Other technic
//df.toDF(col_names: _*).drop(col("fnlwgt")).na.drop("any").show(3,1000, true)

-RECORD 0----------------------------
 age            | 39                 
 workclass      | State-gov          
 education      | Bachelors          
 education-num  | 13                 
 marital-status | Never-married      
 occupation     | Adm-clerical       
 relationship   | Not-in-family      
 race           | White              
 sex            | Male               
 capital-gain   | 2174               
 capital-loss   | 0                  
 hours-per-week | 40                 
 native-country | United-States      
 earnings       | <=50K              
-RECORD 1----------------------------
 age            | 50                 
 workclass      | Self-emp-not-inc   
 education      | Bachelors          
 education-num  | 13                 
 marital-status | Married-civ-spouse 
 occupation     | Exec-managerial    
 relationship   | Husband            
 race           | White              
 sex            | Male               
 capital-gain   | 0                  
 capital-los

column_replacing: Array[org.apache.spark.sql.Column] = Array(_c0 AS `age`, _c1 AS `workclass`, _c2 AS `fnlwgt`, _c3 AS `education`, _c4 AS `education-num`, _c5 AS `marital-status`, _c6 AS `occupation`, _c7 AS `relationship`, _c8 AS `race`, _c9 AS `sex`, _c10 AS `capital-gain`, _c11 AS `capital-loss`, _c12 AS `hours-per-week`, _c13 AS `native-country`, _c14 AS `earnings`)
df: org.apache.spark.sql.DataFrame = [age: int, workclass: string ... 12 more fields]


> **ZADANIA**:
1. Oblicz średnią `capital-gain` oraz `capital-loss` dla każdej narodowości - wynik uporządkuj malejąco po średnim zysku
2. Oblicz jaki procent kobiet zarabia ponad 50K w podziale na wykształcenie - wynik zaokrąglij do 2 miejsc po przecinku
3. Oblicz ile godzin średnio w ciągu tygodnia pracują osoby `Never-married` w stosunku do reszty społeczeństwa
4. W jakim wieku najczęściej zarabia sie powyżej 50K?
5. W podziale ze względu na zmienną `race` oblicz stosunek liczby kobiet do mężczyzn
6. Oblicz różnicę pomiędzy średnim czasem pracy osób uzyskujących `capital-gain` a resztą populacji
7. Oblicz róznicę wieku do średniego wieku osób z danym poziomem wykształcenia

In [62]:
df.createOrReplaceTempView("df")

In [73]:
//1
df.groupBy("native-country").agg(avg("capital-gain").as("Avg_capital_gain"),
                                avg("capital-loss").as("Avg_capital_loss")
                                ).orderBy(col("Avg_capital_gain").desc_nulls_last).show()

+------------------+------------------+------------------+
|    native-country|  Avg_capital_gain|  Avg_capital_loss|
+------------------+------------------+------------------+
|             India|           3604.92|            131.12|
|             Japan|2546.2258064516127| 57.53225806451613|
|            Taiwan|2147.0392156862745| 154.8235294117647|
|              Iran|2044.6511627906978|126.81395348837209|
|             South|         1869.7375|           169.425|
|                 ?|1806.5180102915951|118.26586620926244|
|            Canada|1504.1322314049587|129.93388429752065|
|Dominican-Republic| 1476.057142857143| 25.17142857142857|
|            Greece| 1283.896551724138|295.44827586206895|
|     United-States|1089.2299280082277| 88.51059307507714|
|          Cambodia| 1027.842105263158|183.05263157894737|
|       Philippines|1018.3434343434343| 98.95454545454545|
|           Germany| 887.0948905109489| 77.97810218978103|
|           England| 864.7222222222222| 82.7555555555555

In [94]:
//2
df.filter($"sex" === "Female").withColumn("earnings_flag", when($"earnings" === ">50K", 1).otherwise(0)
                                         ).groupBy("education").agg(round(avg("earnings_flag"), 2).as("percent")).show()

+------------+-------+
|   education|percent|
+------------+-------+
|     Masters|   0.33|
|        10th|   0.01|
|     5th-6th|   0.02|
|  Assoc-acdm|   0.13|
|   Assoc-voc|   0.13|
|     7th-8th|   0.01|
|         9th|   0.03|
|     HS-grad|   0.07|
|   Bachelors|   0.21|
|        11th|   0.02|
|     1st-4th|    0.0|
|   Preschool|    0.0|
|        12th|   0.03|
|   Doctorate|   0.58|
|Some-college|   0.07|
| Prof-school|   0.46|
+------------+-------+



In [93]:
//2 sql approach
val q1 = """(select education, round(avg(earnings), 2) percent from 
     (select sex, education, case when earnings = '>50K' then 1 else 0 end earnings from df where sex = 'Female')
      group by education)"""
spark.sql(q1).show()

+------------+-------+
|   education|percent|
+------------+-------+
|     Masters|   0.33|
|        10th|   0.01|
|     5th-6th|   0.02|
|  Assoc-acdm|   0.13|
|   Assoc-voc|   0.13|
|     7th-8th|   0.01|
|         9th|   0.03|
|     HS-grad|   0.07|
|   Bachelors|   0.21|
|        11th|   0.02|
|     1st-4th|    0.0|
|   Preschool|    0.0|
|        12th|   0.03|
|   Doctorate|   0.58|
|Some-college|   0.07|
| Prof-school|   0.46|
+------------+-------+



q1: String =
(select education, round(avg(earnings), 2) percent from
     (select sex, education, case when earnings = '>50K' then 1 else 0 end earnings from df where sex = 'Female')
      group by education)


In [103]:
//3. Oblicz ile godzin średnio w ciągu tygodnia pracują osoby `Never-married` w stosunku do reszty społeczeństwa

df.select($"hours-per-week",
         (when($"marital-status" === "Never-married", "Never-married").otherwise("Rest")).as("flag")
         ).groupBy("flag").avg().show()

+-------------+-------------------+
|         flag|avg(hours-per-week)|
+-------------+-------------------+
|Never-married|   36.9399981278667|
|         Rest|  42.14526007861779|
+-------------+-------------------+



In [107]:
//4. W jakim wieku najczęściej zarabia sie powyżej 50K?

df.select($"age",
          when($"earnings" === ">50K", 1).otherwise(0).as("earnings")
         ).groupBy("age").agg(avg("earnings").as("avg")).orderBy($"avg".desc).show(3)

+---+-------------------+
|age|                avg|
+---+-------------------+
| 50| 0.4335548172757475|
| 54|0.41686746987951806|
| 79| 0.4090909090909091|
| 53| 0.4073275862068966|
| 47| 0.4067796610169492|
| 51|0.40672268907563025|
| 52|  0.401673640167364|
| 48| 0.3996316758747698|
| 46|0.39620081411126185|
| 45| 0.3923705722070845|
| 44|0.38812154696132595|
| 59|0.37464788732394366|
| 57| 0.3659217877094972|
| 49|0.35701906412478335|
| 43|0.35454545454545455|
| 55|0.34844868735083534|
| 42|0.34615384615384615|
| 41|0.34529702970297027|
| 38|0.34099153567110035|
| 39|0.34068627450980393|
+---+-------------------+
only showing top 20 rows



In [111]:
//5. W podziale ze względu na zmienną `race` oblicz stosunek liczby kobiet do mężczyzn

df.select($"race",
         when($"sex" === "Male", 1).otherwise(0).as("M"),
         when($"sex" === "Female", 1).otherwise(0).as("F")
         ).groupBy("race"
         ).agg(round(sum("F")/sum("M"),2).as("ratio")).show()

+------------------+-----+
|              race|ratio|
+------------------+-----+
|             Other| 0.67|
|Amer-Indian-Eskimo| 0.62|
|             White| 0.45|
|Asian-Pac-Islander|  0.5|
|             Black| 0.99|
+------------------+-----+



In [139]:
//6. Oblicz różnicę pomiędzy średnim czasem pracy osób uzyskujących `capital-gain` a resztą populacji

val x = df.select($"hours-per-week",
         when($"capital-gain" > 0, "gain").otherwise("rest").as("flag")
         ).groupBy("flag").agg(avg("hours-per-week").as("avg_hours"))

x.show()

+----+------------------+
|flag|         avg_hours|
+----+------------------+
|gain| 43.51032448377581|
|rest|40.158263258400616|
+----+------------------+



x: org.apache.spark.sql.DataFrame = [flag: string, avg_hours: double]


In [140]:
val rows = x.collect()

rows: Array[org.apache.spark.sql.Row] = Array([gain,43.51032448377581], [rest,40.158263258400616])


In [138]:
rows(0).get(1).asInstanceOf[Double] - rows(1).get(1).asInstanceOf[Double]

res109: Double = 3.352061225375195


In [10]:
import org.apache.spark.sql.expressions.Window

import org.apache.spark.sql.expressions.Window


In [155]:
// 7. Oblicz róznicę wieku do średniego wieku osób z danym poziomem wykształcenia

val wndw = Window.partitionBy("education")

df.select($"education", $"age", 
          avg("age").over(wndw).alias("avg_edu_age")
         ).withColumn("diff",   $"age" - $"avg_edu_age").show()


+---------+---+-----------------+-------------------+
|education|age|      avg_edu_age|               diff|
+---------+---+-----------------+-------------------+
|     10th| 67|37.42979635584137|  29.57020364415863|
|     10th| 17|37.42979635584137| -20.42979635584137|
|     10th| 25|37.42979635584137|-12.429796355841368|
|     10th| 60|37.42979635584137|  22.57020364415863|
|     10th| 59|37.42979635584137|  21.57020364415863|
|     10th| 55|37.42979635584137|  17.57020364415863|
|     10th| 36|37.42979635584137|-1.4297963558413684|
|     10th| 33|37.42979635584137| -4.429796355841368|
|     10th| 27|37.42979635584137|-10.429796355841368|
|     10th| 58|37.42979635584137|  20.57020364415863|
|     10th| 52|37.42979635584137| 14.570203644158632|
|     10th| 17|37.42979635584137| -20.42979635584137|
|     10th| 49|37.42979635584137| 11.570203644158632|
|     10th| 58|37.42979635584137|  20.57020364415863|
|     10th| 32|37.42979635584137| -5.429796355841368|
|     10th| 17|37.4297963558

wndw: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@6e99ec9f


----

## SalaryHistory

In [1]:
val random = new scala.util.Random(123)
val years = 6
val names = List("Alice", "Betty", "Chris", "Dan", "Greg")
val names_count = names.length

val names_list = (1 to years).flatMap(x => names.map(y => y)).sorted

val year = (1 to names_count).flatMap(x => (0 until years).map(y => 2005 + y))

val starting_salary = (1 to names_count).map(x => {val n = random.nextGaussian()*1000 + 4000; 
                                                   BigDecimal(n).setScale(2, BigDecimal.RoundingMode.HALF_UP
                                                                         ).toDouble})

var temp_salary = starting_salary.flatMap(x => x :: List.fill(years-1)(0.0)).toList

Intitializing Scala interpreter ...

Spark Web UI available at http://ce7d2534405e:4040
SparkContext available as 'sc' (version = 3.0.0, master = local[*], app id = local-1597906445570)
SparkSession available as 'spark'


random: scala.util.Random = scala.util.Random@6418e423
years: Int = 6
names: List[String] = List(Alice, Betty, Chris, Dan, Greg)
names_count: Int = 5
names_list: scala.collection.immutable.IndexedSeq[String] = Vector(Alice, Alice, Alice, Alice, Alice, Alice, Betty, Betty, Betty, Betty, Betty, Betty, Chris, Chris, Chris, Chris, Chris, Chris, Dan, Dan, Dan, Dan, Dan, Dan, Greg, Greg, Greg, Greg, Greg, Greg)
year: scala.collection.immutable.IndexedSeq[Int] = Vector(2005, 2006, 2007, 2008, 2009, 2010, 2005, 2006, 2007, 2008, 2009, 2010, 2005, 2006, 2007, 2008, 2009, 2010, 2005, 2006, 2007, 2008, 2009, 2010, 2005, 2006, 2007, 2008, 2009, 2010)
starting_salary: scala.collection.immutable.IndexedSeq[Double] = Vector(2561.95, 4634.2, 4226.06, 4277.46, 4184.32)
temp_salary: List[Double] = List(2...


In [2]:
def replaceZero(l: List[Double]): List[Double]= {
    var diff = random.nextGaussian()*0.09 + 0.1;
    l match {
      case x :: y :: tail if x != 0.0 & y !=0 => x :: replaceZero(y :: tail)
      case x :: 0.0 :: tail if x != 0.0 => x :: replaceZero((x * (1+diff)):: tail)
      case x :: Nil  if x != 0.0 => x :: Nil
    }
}

def roundToTwoDec(l: List[Double]) = 
            l.map(x => BigDecimal(x).setScale(2, BigDecimal.RoundingMode.HALF_UP).toDouble)

def zipMany[X,Y,Z](xs: Seq[X], ys: Seq[Y], zs: Seq[Z]): Seq[(X,Y,Z)] =
    for (((x, y), z) <- xs zip ys zip zs) yield (x, y, z)

replaceZero: (l: List[Double])List[Double]
roundToTwoDec: (l: List[Double])List[Double]
zipMany: [X, Y, Z](xs: Seq[X], ys: Seq[Y], zs: Seq[Z])Seq[(X, Y, Z)]


In [4]:
val salary = roundToTwoDec(replaceZero(temp_salary))

val salary_history = zipMany(names_list, year, salary).toDF("name", "year", "salary")

val sh = salary_history.filter(!($"name" === "Greg" && $"year" === 2006)).union(
           Seq(("Alice", 2006, 3000.00): Tuple3[String, Int, Double]).toDF("name", "year", "salary"))

salary: List[Double] = List(2561.95, 2733.94, 3340.0, 3781.99, 4090.31, 4873.91, 4634.2, 4309.26, 4559.02, 4739.57, 4497.16, 4838.16, 4226.06, 4107.93, 4553.96, 4849.8, 5834.7, 7004.38, 4277.46, 5398.69, 6652.84, 7960.74, 9667.38, 8975.86, 4184.32, 4624.18, 5289.15, 5148.91, 5158.04, 6175.91)
salary_history: org.apache.spark.sql.DataFrame = [name: string, year: int ... 1 more field]
sh: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [name: string, year: int ... 1 more field]


In [5]:
val sh = salary_history.filter(!($"name" === "Greg" && $"year" === 2006)).union(
           Seq(("Alice", 2006, 3000.00): Tuple3[String, Int, Double]).toDF("name", "year", "salary"))

sh: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [name: string, year: int ... 1 more field]


In [6]:
sh.show(30)

+-----+----+-------+
| name|year| salary|
+-----+----+-------+
|Alice|2005|2561.95|
|Alice|2006|2733.94|
|Alice|2007| 3340.0|
|Alice|2008|3781.99|
|Alice|2009|4090.31|
|Alice|2010|4873.91|
|Betty|2005| 4634.2|
|Betty|2006|4309.26|
|Betty|2007|4559.02|
|Betty|2008|4739.57|
|Betty|2009|4497.16|
|Betty|2010|4838.16|
|Chris|2005|4226.06|
|Chris|2006|4107.93|
|Chris|2007|4553.96|
|Chris|2008| 4849.8|
|Chris|2009| 5834.7|
|Chris|2010|7004.38|
|  Dan|2005|4277.46|
|  Dan|2006|5398.69|
|  Dan|2007|6652.84|
|  Dan|2008|7960.74|
|  Dan|2009|9667.38|
|  Dan|2010|8975.86|
| Greg|2005|4184.32|
| Greg|2007|5289.15|
| Greg|2008|5148.91|
| Greg|2009|5158.04|
| Greg|2010|6175.91|
|Alice|2006| 3000.0|
+-----+----+-------+



> **ZADANIA**:
1. Oblicz ile razy powtarza się każde z imion w salaryHistory
2. Oblicz średnią, minimum i maksimum z pensji w każdym roku, lata posortuj malejąco, wyniki zaokrąglij do pełnych wartości
3. Oblicz różnicę między pensją w danym roku a średnią pensją danej osoby na przestrzeni wszystkich lat
4. Porównaj pensje ludzi pomiedzy najwcześniejszym i najpóźniejszym rokiem ich pracy
5. Czy komukolwiek obniżyła się pensja w stosunku do roku poprzedniego? **a.** Ile osób było kiedykolwiek w takiej sytuacji? **b.** Jaki jest rozkład częstości takich przypadków w zależności od roku?
6. Oblicz różnicę w pensjach w stosuku do: **a.** najwyższej pensji danej osoby, **b.** drugiej najwyższej pensji danej osoby
7. Oblicz medianę wszystkich zarobków
8. Oblicz medianę zarobków poszczególnych osób

In [7]:
//1. Oblicz ile razy powtarza się każde z imion w salaryHistor

sh.groupBy($"name").count().show()

+-----+-----+
| name|count|
+-----+-----+
|Chris|    6|
| Greg|    5|
|Betty|    6|
|  Dan|    6|
|Alice|    7|
+-----+-----+



In [8]:
//2. Oblicz średnią, minimum i maksimum z pensji w każdym roku, lata posortuj malejąco, 
//wyniki zaokrąglij do pełnych wartości

sh.groupBy($"year").agg(
                        min("salary").as("min"),
                        max("salary").as("max"),
                        round(avg("salary"),2).as("avg")).show()

+----+-------+-------+-------+
|year|    min|    max|    avg|
+----+-------+-------+-------+
|2007| 3340.0|6652.84|4878.99|
|2006|2733.94|5398.69|3909.96|
|2009|4090.31|9667.38|5849.52|
|2005|2561.95| 4634.2| 3976.8|
|2010|4838.16|8975.86|6373.64|
|2008|3781.99|7960.74| 5296.2|
+----+-------+-------+-------+



In [13]:
//3. Oblicz różnicę między pensją w danym roku a średnią pensją danej osoby na przestrzeni wszystkich lat

val avg_sal_wndw = Window.partitionBy("name")

sh.select($"name", $"year", $"salary",
         round(avg("salary").over(avg_sal_wndw),2).as("avg_salary")
         ).withColumn("diff", round($"avg_salary" - $"salary"))
.show()

+-----+----+-------+----------+-------+
| name|year| salary|avg_salary|   diff|
+-----+----+-------+----------+-------+
|Chris|2005|4226.06|   5096.14|  870.0|
|Chris|2006|4107.93|   5096.14|  988.0|
|Chris|2007|4553.96|   5096.14|  542.0|
|Chris|2008| 4849.8|   5096.14|  246.0|
|Chris|2009| 5834.7|   5096.14| -739.0|
|Chris|2010|7004.38|   5096.14|-1908.0|
| Greg|2005|4184.32|   5191.27| 1007.0|
| Greg|2007|5289.15|   5191.27|  -98.0|
| Greg|2008|5148.91|   5191.27|   42.0|
| Greg|2009|5158.04|   5191.27|   33.0|
| Greg|2010|6175.91|   5191.27| -985.0|
|Betty|2005| 4634.2|   4596.23|  -38.0|
|Betty|2006|4309.26|   4596.23|  287.0|
|Betty|2007|4559.02|   4596.23|   37.0|
|Betty|2008|4739.57|   4596.23| -143.0|
|Betty|2009|4497.16|   4596.23|   99.0|
|Betty|2010|4838.16|   4596.23| -242.0|
|  Dan|2005|4277.46|    7155.5| 2878.0|
|  Dan|2006|5398.69|    7155.5| 1757.0|
|  Dan|2007|6652.84|    7155.5|  503.0|
+-----+----+-------+----------+-------+
only showing top 20 rows



avg_sal_wndw: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@6bfba0b8


In [11]:
//4. Porównaj pensje ludzi pomiedzy najwcześniejszym i najpóźniejszym rokiem ich pracy

// uśredniona pensja Alice w 2006 roku
val salaryHistory_avg = sh.groupBy("name", "year").agg(avg("salary").alias("salary"))

val asc_wndw = Window.partitionBy("name").orderBy("year")
val desc_wndw = Window.partitionBy("name").orderBy(col("year").desc)

salaryHistory_avg.withColumn("diff", first("salary").over(desc_wndw) - first("salary").over(asc_wndw)
                            ).select($"name", round($"diff").as("diff")).distinct().show()



+-----+------+
| name|  diff|
+-----+------+
|Chris|2778.0|
| Greg|1992.0|
|Betty| 204.0|
|  Dan|4698.0|
|Alice|2312.0|
+-----+------+



salaryHistory_avg: org.apache.spark.sql.DataFrame = [name: string, year: int ... 1 more field]
asc_wndw: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@57836e69
desc_wndw: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@5ca7f806


In [14]:
//5. Czy komukolwiek obniżyła się pensja w stosunku do roku poprzedniego? 
//**a.** Ile osób było kiedykolwiek w takiej sytuacji? 
//**b.** Jaki jest rozkład częstości takich przypadków w zależności od roku?

val wnd_prev = Window.partitionBy("name").orderBy("year")

sh.withColumn("prev_salary", lag("salary", 1).over(wnd_prev)
             ).withColumn("diff", $"salary" - $"prev_salary").show()

+-----+----+-------+-----------+-------------------+
| name|year| salary|prev_salary|               diff|
+-----+----+-------+-----------+-------------------+
|Chris|2005|4226.06|       null|               null|
|Chris|2006|4107.93|    4226.06|-118.13000000000011|
|Chris|2007|4553.96|    4107.93| 446.02999999999975|
|Chris|2008| 4849.8|    4553.96| 295.84000000000015|
|Chris|2009| 5834.7|     4849.8|  984.8999999999996|
|Chris|2010|7004.38|     5834.7| 1169.6800000000003|
| Greg|2005|4184.32|       null|               null|
| Greg|2007|5289.15|    4184.32|            1104.83|
| Greg|2008|5148.91|    5289.15|-140.23999999999978|
| Greg|2009|5158.04|    5148.91|   9.13000000000011|
| Greg|2010|6175.91|    5158.04| 1017.8699999999999|
|Betty|2005| 4634.2|       null|               null|
|Betty|2006|4309.26|     4634.2| -324.9399999999996|
|Betty|2007|4559.02|    4309.26| 249.76000000000022|
|Betty|2008|4739.57|    4559.02| 180.54999999999927|
|Betty|2009|4497.16|    4739.57|-242.409999999

wnd_prev: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@30e03ed8


In [15]:
//a
sh.withColumn("prev_salary", lag("salary", 1).over(wnd_prev)
             ).withColumn("diff", $"salary" - $"prev_salary"
             ).filter($"diff" < 0).show()

+-----+----+-------+-----------+-------------------+
| name|year| salary|prev_salary|               diff|
+-----+----+-------+-----------+-------------------+
|Chris|2006|4107.93|    4226.06|-118.13000000000011|
| Greg|2008|5148.91|    5289.15|-140.23999999999978|
|Betty|2006|4309.26|     4634.2| -324.9399999999996|
|Betty|2009|4497.16|    4739.57|-242.40999999999985|
|  Dan|2010|8975.86|    9667.38| -691.5199999999986|
+-----+----+-------+-----------+-------------------+



In [16]:
//b
sh.withColumn("flag", 
             when(($"salary" - lag($"salary",1).over(wnd_prev)) < 0 , 1).otherwise(0)
             ).groupBy("year").agg(sum("flag").as("n_cases")).orderBy("year").show()

+----+-------+
|year|n_cases|
+----+-------+
|2005|      0|
|2006|      2|
|2007|      0|
|2008|      1|
|2009|      1|
|2010|      1|
+----+-------+



In [17]:
//6. Oblicz różnicę w pensjach w stosuku do: 
//**a.** najwyższej pensji danej osoby,
//**b.** drugiej najwyższej pensji danej osoby

val max_sal = sh.select($"name", $"salary").groupBy("name").agg(max("salary").as("max_salary"))

//a
sh.join(max_sal, sh.col("name")=== max_sal.col("name"), "inner").drop(max_sal.col("name")
        ).withColumn("diff", $"max_salary" - $"salary").show()

+----+-------+-----+----------+------------------+
|year| salary| name|max_salary|              diff|
+----+-------+-----+----------+------------------+
|2005|2561.95|Alice|   4873.91|           2311.96|
|2006|2733.94|Alice|   4873.91|           2139.97|
|2007| 3340.0|Alice|   4873.91|1533.9099999999999|
|2008|3781.99|Alice|   4873.91|           1091.92|
|2009|4090.31|Alice|   4873.91| 783.5999999999999|
|2010|4873.91|Alice|   4873.91|               0.0|
|2005| 4634.2|Betty|   4838.16|203.96000000000004|
|2006|4309.26|Betty|   4838.16| 528.8999999999996|
|2007|4559.02|Betty|   4838.16| 279.1399999999994|
|2008|4739.57|Betty|   4838.16| 98.59000000000015|
|2009|4497.16|Betty|   4838.16|             341.0|
|2010|4838.16|Betty|   4838.16|               0.0|
|2005|4226.06|Chris|   7004.38|2778.3199999999997|
|2006|4107.93|Chris|   7004.38|           2896.45|
|2007|4553.96|Chris|   7004.38|           2450.42|
|2008| 4849.8|Chris|   7004.38|           2154.58|
|2009| 5834.7|Chris|   7004.38|

max_sal: org.apache.spark.sql.DataFrame = [name: string, max_salary: double]


In [29]:
//b 
val max_sal_wndw = Window.partitionBy("name").orderBy(col("salary").desc)

sh
.withColumn("highest", first("salary").over(max_sal_wndw)) 
.withColumn("secondHigh", first(lead("salary", 1).over(max_sal_wndw)).over(max_sal_wndw))
.withColumn("diffHigh", $"salary" - first("salary").over(max_sal_wndw)) 
.withColumn("diffScndHigh", $"salary" - first(lead("salary", 1).over(max_sal_wndw)).over(max_sal_wndw)) 
.withColumn("lead1", lead("salary", 1).over(max_sal_wndw))
.withColumn("lag1", lag($"salary",1).over(max_sal_wndw))

.show(6)

+-----+----+-------+-------+----------+-------------------+-------------------+-------+-------+
| name|year| salary|highest|secondHigh|           diffHigh|       diffScndHigh|  lead1|   lag1|
+-----+----+-------+-------+----------+-------------------+-------------------+-------+-------+
|Chris|2010|7004.38|7004.38|    5834.7|                0.0| 1169.6800000000003| 5834.7|   null|
|Chris|2009| 5834.7|7004.38|    5834.7|-1169.6800000000003|                0.0| 4849.8|7004.38|
|Chris|2008| 4849.8|7004.38|    5834.7|           -2154.58| -984.8999999999996|4553.96| 5834.7|
|Chris|2007|4553.96|7004.38|    5834.7|           -2450.42|-1280.7399999999998|4226.06| 4849.8|
|Chris|2005|4226.06|7004.38|    5834.7|-2778.3199999999997|-1608.6399999999994|4107.93|4553.96|
|Chris|2006|4107.93|7004.38|    5834.7|           -2896.45|-1726.7699999999995|   null|4226.06|
+-----+----+-------+-------+----------+-------------------+-------------------+-------+-------+
only showing top 6 rows



max_sal_wndw: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@6005a3c3


In [31]:
//7. Oblicz medianę wszystkich zarobków

sh.stat.approxQuantile("salary", Array(0.5), 0.001)

res21: Array[Double] = Array(4634.2)


In [32]:
//8. Oblicz medianę zarobków poszczególnych osób

val wndw_1 = Window.partitionBy("name").orderBy("salary")
val wndw_2 = Window.partitionBy("name")

sh.withColumn("rank", row_number().over(wndw_1))
.withColumn("avg", avg(col("rank")).over(wndw_2))
.withColumn("helper", when(abs(col("rank") - col("avg")) < 1, col("salary")).otherwise(null))
.withColumn("median", avg(col("helper")).over(wndw_2)).show(30) 

+-----+----+-------+----+---+-------+-----------------+
| name|year| salary|rank|avg| helper|           median|
+-----+----+-------+----+---+-------+-----------------+
|Chris|2006|4107.93|   1|3.5|   null|          4701.88|
|Chris|2005|4226.06|   2|3.5|   null|          4701.88|
|Chris|2007|4553.96|   3|3.5|4553.96|          4701.88|
|Chris|2008| 4849.8|   4|3.5| 4849.8|          4701.88|
|Chris|2009| 5834.7|   5|3.5|   null|          4701.88|
|Chris|2010|7004.38|   6|3.5|   null|          4701.88|
| Greg|2005|4184.32|   1|3.0|   null|          5158.04|
| Greg|2008|5148.91|   2|3.0|   null|          5158.04|
| Greg|2009|5158.04|   3|3.0|5158.04|          5158.04|
| Greg|2007|5289.15|   4|3.0|   null|          5158.04|
| Greg|2010|6175.91|   5|3.0|   null|          5158.04|
|Betty|2006|4309.26|   1|3.5|   null|4596.610000000001|
|Betty|2009|4497.16|   2|3.5|   null|4596.610000000001|
|Betty|2007|4559.02|   3|3.5|4559.02|4596.610000000001|
|Betty|2005| 4634.2|   4|3.5| 4634.2|4596.610000

wndw_1: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@630b8eab
wndw_2: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@79c6204e


In [33]:
sh.groupBy("name").agg(expr("percentile(salary, 0.5) as median")).show()

+-----+-----------------+
| name|           median|
+-----+-----------------+
|Chris|          4701.88|
| Greg|          5158.04|
|Betty|4596.610000000001|
|  Dan|          7306.79|
|Alice|           3340.0|
+-----+-----------------+

